# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
# !pip install wandb -qU

In [2]:
# 필수 라이브러리 및 모듈 임포트
import pandas as pd  # 데이터 처리
import os  # 시스템 경로 및 파일 관리
import re  # 정규 표현식을 사용한 문자열 처리
import json  # JSON 데이터 처리
import yaml  # YAML 파일 처리
from glob import glob  # 파일 경로 패턴 매칭
from tqdm import tqdm  # 진행률 표시 바
from pprint import pprint  # 데이터 구조 예쁘게 출력
import torch  # PyTorch 기반의 딥러닝 연산
import pytorch_lightning as pl  # PyTorch를 좀 더 쉽게 사용하기 위한 확장 라이브러리
from rouge import Rouge  # 자동 텍스트 요약 성능 평가를 위한 ROUGE 지표 계산

# 데이터 로딩 및 처리를 위한 PyTorch 유틸리티
from torch.utils.data import Dataset, DataLoader  
# 트랜스포머 모델 (여기서는 BART) 사용을 위한 Hugging Face 라이브러리
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig  
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer  # Seq2Seq 모델 학습 설정 및 트레이너
from transformers import Trainer, TrainingArguments  # 일반 모델 학습 설정 및 트레이너
from transformers import EarlyStoppingCallback  # 학습 과정의 조기 종료를 위한 콜백

# Weights & Biases를 사용한 학습 프로세스 추적 및 시각화
import wandb  
wandb.login()  # Weights & Biases 계정으로 로그인

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dbsrua1105 (dubiduwab). Use `wandb login --relogin` to force relogin


True

In [3]:
# 열의 최대 너비 설정
pd.set_option('display.max_colwidth', 100)

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [4]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [5]:
MAX_LENGTH = 100

In [6]:
# Dialogue Summarization을 위한 모델 학습, 평가 및 예측 설정을 정의하는 config 데이터
config_data = {
    "general": {  # 일반 설정
        "data_path": "../data/",  # 데이터가 위치한 경로
        "model_name": "digit82/kobart-summarization",  # 사용할 사전 학습된 모델 이름
        "output_dir": "./result/checkpoint/"  # 학습된 모델과 결과물을 저장할 경로
    },
    "tokenizer": {  # 토크나이저 설정
        "encoder_max_len": 512,  # 인코더 최대 길이
        "decoder_max_len": 100,  # 디코더 최대 길이
        "bos_token": f"{tokenizer.bos_token}",  # 문장 시작 토큰
        "eos_token": f"{tokenizer.eos_token}",  # 문장 종료 토큰
        # "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']  # 토큰화 과정에서 무시될 특수 토큰
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#PassportNumber#', '#PhoneNumber#', '#SSN#', '#주제#']
    },
    "training": {  # 학습 설정
        "overwrite_output_dir": True,  # 출력 디렉토리 덮어쓰기 여부
        "num_train_epochs": 20,  # 학습 에포크 수
        "learning_rate": 5e-5,  # 학습률
        "per_device_train_batch_size": 32,  # 학습 배치 크기
        "per_device_eval_batch_size": 32,  # 평가 배치 크기
        "warmup_ratio": 0.1,  # 워밍업 비율
        "weight_decay": 0.01,  # 가중치 감소
        "lr_scheduler_type": 'cosine',  # 학습률 스케줄러 유형
        "optim": 'adamw_torch',  # 옵티마이저
        "gradient_accumulation_steps": 1,  # 그래디언트 누적 스텝
        "evaluation_strategy": 'epoch',  # 평가 전략
        "save_strategy": 'epoch',  # 저장 전략
        "save_total_limit": 5,  # 최대 저장 파일 수
        "fp16": True,  # 반정밀도(float16) 학습 사용 여부
        "load_best_model_at_end": True,  # 학습 종료 시 최고 성능 모델 로드 여부
        "seed": 42,  # 랜덤 시드
        "logging_dir": "./logs",  # 로깅 디렉토리
        "logging_strategy": "epoch",  # 로깅 전략
        "predict_with_generate": True,  # 생성을 통한 예측 사용 여부
        # "generation_max_length": 100,  # 생성 최대 길이
        "generation_max_length": MAX_LENGTH,
        "do_train": True,  # 학습 수행 여부
        "do_eval": True,  # 평가 수행 여부
        "early_stopping_patience": 3,  # 조기 종료 인내심
        "early_stopping_threshold": 0.0005,  # 조기 종료 임계값
        "report_to": "wandb"  # 학습 프로세스를 Weights & Biases에 보고
    },
    "wandb": {  # Weights & Biases 설정
        "entity": "wandb_repo",
        "project": "project_name",
        "name": "run_name"
    },
    "inference": {  # 예측 설정
        "ckt_path": "./",  # 체크포인트 경로
        "result_path": "./result/output/",  # 결과물 저장 경로
        "no_repeat_ngram_size": 2,  # 반복 n-gram 크기
        "early_stopping": True,  # 예측 시 조기 종료 사용 여부
        # "generate_max_length": 100,  # 생성 최대 길이
        "generate_max_length": MAX_LENGTH,
        "num_beams": 4,  # 문장 생성시 다음 단어를 탐색하는 영역의 개수
        "batch_size" : 32,  # 배치 크기
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]  # 제거할 토큰
    }
}


- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [7]:
# 모델 구성 설정을 YAML 파일로 저장
config_path = "./config.yaml"  # 설정 파일 경로 지정
# 설정 파일을 쓰기 모드로 열고, config_data를 YAML 형식으로 저장합니다.
with open(config_path, "w") as file:
    # allow_unicode=True 옵션으로 유니코드 문자 정상 처리
    yaml.dump(config_data, file, allow_unicode=True) 


### 3) Configuration 불러오기

In [8]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

# 설정 파일을 읽기 모드로 열고 YAML 형식의 내용을 파이썬 객체로 변환합니다.
with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '../data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './result/checkpoint/'},
 'inference': {'batch_size': 32,
               'ckt_path': './',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './result/output/'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 100,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#Person4#',
                                  '#Person5#',
                                  '#Person6#',
                                  '#Person7#',
                                  

In [9]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': '../data/',
 'model_name': 'digit82/kobart-summarization',
 'output_dir': './result/checkpoint/'}

In [10]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

In [11]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'decoder_max_len': 100,
 'encoder_max_len': 512,
 'eos_token': '</s>',
 'special_tokens': ['#Person1#',
  '#Person2#',
  '#Person3#',
  '#Person4#',
  '#Person5#',
  '#Person6#',
  '#Person7#',
  '#Address#',
  '#CarNumber#',
  '#CardNumber#',
  '#DateOfBirth#',
  '#Email#',
  '#PassportNumber#',
  '#PhoneNumber#',
  '#SSN#',
  '#주제#']}

In [12]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.0005,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 1,
 'learning_rate': 5e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'linear',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 32,
 'per_device_train_batch_size': 32,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [13]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
# loaded_config['wandb']

In [14]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "nlp-advanced"
loaded_config['wandb']['name'] = "YUNGYEOM KIM"
loaded_config['wandb']['project'] = "Dialogue-Summarization"

pprint(loaded_config['wandb'])

{'entity': 'nlp-advanced',
 'name': 'YUNGYEOM KIM',
 'project': 'Dialogue-Summarization'}


In [15]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': './',
 'early_stopping': True,
 'generate_max_length': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
 'result_path': './result/output/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [16]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train_topic.csv'))
train_df.tail()

,fname,dialogue,summary,topic,dialogue_with_topic
12452,train_12455,"#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2#: 네, 저를 알아봐 주셔서 기쁩니다.\n#Person1#: 텔렉스에서 자신을 백발과 수...",탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다. 탄은 그린 씨를 위해 호텔에서 큰 스위트룸을 예약했습니다.,누군가를 태우다,"#주제#: 누군가를 태우다\n#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2#: 네, 저를 알아봐 주셔서 기쁩니다.\n#Person1#:..."
12453,train_12456,"#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 했죠?\n#Person2#: 네, 그는 특히 우리가 늦지 않도록 요청했습니다. 우리 동요크...",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼런스 센터로 지하철을 같이 타기로 계획했습니다.,컨퍼런스 센터,"#주제#: 컨퍼런스 센터\n#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 했죠?\n#Person2#: 네, 그는 특히 우리가 늦지 않도록..."
12454,train_12457,"#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶습니다.\n#Person1#: 찾아볼게요. 대형 차, 중형 차, 소형 차 중에서 선택하실 ...",#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트,"#주제#: 차 렌트\n#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶습니다.\n#Person1#: 찾아볼게요. 대형 차, 중형 차, 소형..."
12455,train_12458,"#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2#: 음, 어제 엄마가 일자리를 잃었어.\n#Person1#: 그런 일이 있었다니 유감스럽...",#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 않기를 바란다. #Person1#은 #Person2#에게 인터넷에서 일자리 정보를 찾아보는...,실직,"#주제#: 실직\n#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2#: 음, 어제 엄마가 일자리를 잃었어.\n#Person1#: 그런 일이..."
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 타고 갈 거예요. 오늘 가방을 싸도 될까요?\n#Person2#: 네, 그게 좋을 것 같아...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 #Person2#에게 조언을 구합니다.,짐 싸기,"#주제#: 짐 싸기\n#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 타고 갈 거예요. 오늘 가방을 싸도 될까요?\n#Person2#: 네..."


In [17]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev_topic.csv'))
val_df.tail()

,fname,dialogue,summary,topic,dialogue_with_topic
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#Person2#: 그래? 드디어 모든 것을 새롭게 시작하려고 결정했어? \r\n#Person...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2#는 #Person1#를 지지합니다.,새해,#주제#: 새해\n#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#Person2#: 그래? 드디어 모든 것을 새롭게 시작하려고 결정했어? \...
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인지 모르겠어. \r\n#Person1#: 네가 사랑에 빠졌다는 걸 기억해. \r\n#Per...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#는 부인했다.,사랑에 빠지다,"#주제#: 사랑에 빠지다\n#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인지 모르겠어. \r\n#Person1#: 네가 사랑에 빠졌다는 ..."
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동안 차에서 이상한 소리가 나서 오늘 한번 봐주실 수 있을까요?\r\n#Person1#: 어...","#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교체해야 할 것으로 생각하지만, #Person1#는 내일까지 그것을 고칠 수 없습니다. #P...",소음,"#주제#: 소음\n#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동안 차에서 이상한 소리가 나서 오늘 한번 봐주실 수 있을까요?\r\n#P..."
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n#Person2#: 안녕하세요, 어제 웹사이트에서 받은 책을 읽다가 한 페이지가 빠져있는 ...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지가 빠져 있다고 합니다. #Person1#은 문제가 확인되면 새 책을 보내드릴 것이라고 말...,빠진 페이지,"#주제#: 빠진 페이지\n#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n#Person2#: 안녕하세요, 어제 웹사이트에서 받은 책을 읽다..."
498,dev_499,"#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#: 응, 알아. 이번 해는 정말 빨리 갔어.\r\n#Person1#: 이번 여름 휴가에 뭐 ...",#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 일할 것이라고 말한다. #Person1#는 그것이 멋진 직업이라고 생각한다.,여름 휴가,"#주제#: 여름 휴가\n#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#: 응, 알아. 이번 해는 정말 빨리 갔어.\r\n#Person1#:..."


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [18]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:
        # 문장의 시작과 끝을 나타내는 토큰을 초기화
        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue_with_topic','summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue_with_topic']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset,is_test = False):
        # 테스트 데이터일 경우, 인코더 입력과 초기 디코더 입력만 생성
        if is_test:
            encoder_input = dataset['dialogue_with_topic']
            decoder_input = [self.bos_token] * len(dataset['dialogue_with_topic'])
            return encoder_input.tolist(), list(decoder_input)
        # 학습 데이터일 경우, 인코더 입력, 디코더 입력, 디코더 출력을 생성
        else:
            encoder_input = dataset['dialogue_with_topic']
            decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
            decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [19]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        # 초기화 시, 인코더 입력, 디코더 입력, 레이블 및 데이터 길이를 저장
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        # 데이터셋에서 특정 인덱스의 데이터를 가져올 때 호출
        # 인코더 입력과 디코더 입력에서 각각 해당 인덱스의 데이터를 추출하고, 'decoder_input_ids'와 'decoder_attention_mask'로 이름을 변경
        # 이렇게 처리된 데이터를 합쳐서 반환
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    # 예측 시에는 레이블이 없으므로, 인코더 입력과 데이터 길이만을 처리
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id # 각 데이터에 대한 식별자
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx] # 데이터 식별자도 함께 반환
        return item

    def __len__(self):
        return self.len


In [20]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train_topic.csv')
    val_file_path = os.path.join(data_path,'dev_topic.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    # 로드된 데이터의 첫 번째 대화 및 요약 출력
    print('-'*150)
    print(f'train_data:\n {train_data["dialogue_with_topic"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue_with_topic"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    # 데이터를 모델 입력 형태로 전처리
    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    # 전처리된 데이터를 토크나이징
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    # 토크나이징된 데이터를 Dataset 객체로 변환
    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [21]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge() # ROUGE 지표 계산을 위한 객체 초기화
    predictions = pred.predictions
    labels = pred.label_ids

    # 레이블에서 무시된 토큰(-100)을 패딩 토큰으로 대체
    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    # 예측값과 레이블을 텍스트로 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    # 디코딩된 예측값과 레이블의 일부를 출력하여 확인
    print('-'*150)
    for i in range(min(3, len(replaced_predictions))):  # 최소 3개 혹은 예측값 개수만큼 반복
        print(f"PRED: {replaced_predictions[i]}")
        print(f"GOLD: {replaced_labels[i]}")
        print('-'*150)
    print('='*150)

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels, avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [22]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # 학습 매개변수 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=config['general']['output_dir'],  # 모델 출력 디렉토리
        overwrite_output_dir=config['training']['overwrite_output_dir'],
        num_train_epochs=config['training']['num_train_epochs'],  # 전체 학습 에폭 수
        learning_rate=config['training']['learning_rate'],  # 학습률
        per_device_train_batch_size=config['training']['per_device_train_batch_size'],  # 학습 시 각 디바이스당 배치 크기
        per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],  # 평가 시 배치 크기
        warmup_ratio=config['training']['warmup_ratio'],  # 학습률 스케줄러의 워밍업 비율
        weight_decay=config['training']['weight_decay'],  # 가중치 감소 정도
        lr_scheduler_type=config['training']['lr_scheduler_type'],  # 학습률 스케줄러 타입
        optim=config['training']['optim'],  # 옵티마이저 종류
        gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],  # 그라디언트 축적 단계 수
        evaluation_strategy=config['training']['evaluation_strategy'],  # 학습 중 평가 전략
        save_strategy=config['training']['save_strategy'],  # 모델 저장 전략
        save_total_limit=config['training']['save_total_limit'],  # 저장할 모델의 최대 수
        fp16=config['training']['fp16'],  # 반정밀도(float16) 사용 여부
        load_best_model_at_end=config['training']['load_best_model_at_end'],  # 학습 종료 시 최고 모델 로드
        seed=config['training']['seed'],  # 랜덤 시드
        logging_dir=config['training']['logging_dir'],  # 로깅 디렉토리
        logging_strategy=config['training']['logging_strategy'],  # 로깅 전략
        predict_with_generate=config['training']['predict_with_generate'],  # 생성을 사용한 예측 활성화
        generation_max_length=config['training']['generation_max_length'],  # 생성 최대 길이
        do_train=config['training']['do_train'],  # 학습 수행 여부
        do_eval=config['training']['do_eval'],  # 평가 수행 여부
        report_to=config['training']['report_to'],  # 학습 프로세스 보고 대상
    )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"]="true"
    os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred), # 성능 평가 지표 계산 함수
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [23]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)

    # 모델 이름으로부터 tokenizer와 모델을 불러옵니다.
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    # config에서 정의된 추가적인 특수 토큰을 tokenizer에 추가
    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 토크나이저에 추가된 특수 토큰으로 인해 변경된 토큰 개수에 맞게 모델의 토큰 임베딩 크기를 조정
    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.

    # 모델을 지정된 디바이스로 이동
    generate_model.to(device)
    print(generate_model.config) # 모델 설정 출력

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [24]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 데이터 전처리를 위한 Preprocess 인스턴스 생성 및 데이터셋 준비
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # 트레이닝을 위한 Trainer 인스턴스 생성 및 학습 시작
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료
    wandb.finish()

In [25]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartConfig {
  "_name_or_path": "digit82/kobart-summarization",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings":

wandb: Currently logged in as: dbsrua1105 (nlp-advanced). Use `wandb login --relogin` to force relogin


---------- Make dataset complete ----------
---------- Make training arguments ----------


...

---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.744800,0.600444,0.360782,0.126531,0.348870
2,0.552900,0.558436,0.379124,0.136423,0.362800
3,0.459800,0.548759,0.389158,0.147468,0.372793
4,0.376100,0.562106,0.383880,0.143039,0.368651
5,0.309000,0.588221,0.376928,0.138287,0.360428
6,0.255600,0.610782,0.376808,0.141568,0.361147


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 감기에 걸리셨는지 물어봅니다. #Person2# 는 천식에 대한 검사를 받을 것입니다.                                                                           
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                              
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   지미는 #Person1# 에게 오후 3시 30분에 헬스장에서 만나자고 제안한다. #Person1# 은 #Person2# 에게 오후 3시 30분에 운동하자고 제안한다.                                                                   
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                        
------------------------------------------------------------------------------------------------

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


eval/loss,▇▂▁▃▅█
eval/rouge-1,▁▆█▇▅▅
eval/rouge-2,▁▄█▇▅▆
eval/rouge-l,▁▅█▇▄▅
eval/runtime,▅▁▂█▄█
eval/samples_per_second,▄█▇▁▅▁
eval/steps_per_second,▄█▇▁▅▁
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███
train/learning_rate,▁█▇▇▆▅
train/loss,█▂▂▁▁▁


In [ ]:
### best moment
# max_length=201
# 8	0.272700	0.559620	0.378479	0.135853	0.362870
# max_length=201 / add special tokens / 
# 8	0.271100	0.558945	0.380258	0.140211	0.364622
# max_length=201 / add special tokens / train_batch=32
# 6	0.281600	0.558865	0.373088	0.132708	0.354238
# output3 / max_length=201 / add special tokens / train_batch=32 / lr=5e-5
# 3	0.284900	0.553649	0.382171	0.138661	0.365527
# output4 / max_length=100 / add special tokens / train_batch=50 / lr=1e-5 / add topic / 
# 8	0.435200	0.552137	0.381672	0.141317	0.367472
# output5_post max_length=100 / add special tokens / train_batch=32 / lr=5e-5 / add topic / 
# 3	0.460600	0.548323	0.394485	0.152015	0.376847
# output6_post max_length=100 / add special tokens / train_batch=32 / lr=5e-5 / add topic / lr_scheduler='linear'
# 3	0.459800	0.548759	0.389158	0.147468	0.372793

## 4. 모델 예측하기

In [26]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "/data/ephemeral/home/code/result/checkpoint/checkpoint-1170"
OUTPUT_NAME = "output6_post.csv"

- test data를 사용하여 모델의 성능을 확인합니다.

In [27]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):
    # 테스트 데이터 파일 경로 설정
    test_file_path = os.path.join(config['general']['data_path'],'test_topic.csv')

    # 테스트 데이터 불러오기 및 데이터프레임 생성
    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    # 로드된 테스트 데이터의 첫 번째 대화 내용 출력
    print('-'*150)
    print(f'test_data:\n{test_data["dialogue_with_topic"][0]}')
    print('-'*150)

    # 테스트 데이터를 모델 입력 형태로 전처리
    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    # 전처리된 데이터를 토크나이징
    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    # 토크나이징된 데이터를 Dataset 객체로 변환
    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    # 전처리 및 토크나이징된 테스트 데이터 및 해당 데이터셋 반환
    return test_data, test_encoder_inputs_dataset

In [28]:
# 예측을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    # 설정에서 모델 이름과 체크포인트 경로를 불러옴
    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)

     # 모델 이름을 사용하여 tokenizer를 로드하고, 설정된 추가 특수 토큰을 추가
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 체크포인트 경로로부터 학습된 모델을 로드
    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)

    # tokenizer에 추가된 특수 토큰의 수에 맞게 모델의 토큰 임베딩 크기를 조절
    generate_model.resize_token_embeddings(len(tokenizer))

    # 모델을 지정된 디바이스로 이동
    generate_model.to(device)
    
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [29]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 예측을 위한 모델과 토크나이저 로드
    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    # 테스트 데이터셋 준비
    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []

    # 모델을 평가 모드로 설정하고, 배치 단위로 요약 생성
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    # 생성된 요약문 정리 및 저장
    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )

    def post_process_summary(summary):
        # 1. '#'과 조사 사이의 띄어쓰기 제거
        summary = re.sub(r'#([A-Za-z0-9_]+)#\s+(은|는|이|가|을|를|에|에게|의|로|으로)', r'#\1#\g<2>', summary)
        # 2. 의미 없는 큰 따옴표 제거
        # summary = summary.replace('"', '')
        summary = summary.replace('"', '')
        # 3. 문장 맨 앞의 들여쓰기 제거
        summary = summary.strip()
        return summary

    # 'summary' 컬럼에 대해 post_process_summary 함수 적용
    output['summary'] = output['summary'].apply(post_process_summary)

    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, OUTPUT_NAME), index=False)

    return output

In [30]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#주제#:   전화 통화  
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요

100%|██████████| 16/16 [00:21<00:00,  1.33s/it]


In [31]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,더슨 씨는 #Person1#에게 사무실 통신은 이메일 통신과 공식 메모로 제한되어 있다고 말한다. #Person2#는 내부와 외부 통신에 적용된다고 설명한다. 그는 또한 새...
1,test_1,"#Person1#은 #Person2#에게 교통 체증에 걸렸다고 말하고, #Person1#이 대중교통을 이용하기 시작하면 환경 오염 문제를 줄일 수 있을 것이라고 말한다."
2,test_2,케이트는 마샤와 히어로가 이혼하려고 한다고 #Person1#에게 말한다. 그녀는 놀라워한다.
3,test_3,#Person1#은 브라이언의 생일을 축하하기 위해 파티에 초대한다. 브라이언이 #Person1#의 목걸이를 칭찬한다.
4,test_4,#Person1# 과 #Person2#는 올림픽 공원에 대해 이야기하고 있습니다. #Person1#은 관람석 5000개의 좌석에 대해 설명합니다.
...,...,...
494,test_495,잭은 찰리에게 공항으로 아빠를 데리러 가야하기 때문에 #Person1# 과 함께 비디오 게임을 하자고 제안한다.
495,test_496,#Person2#는 #Person1#에게 컨트리 음악의 라디오 프로그램에서 일하기 시작했다고 말합니다.
496,test_497,"앨리스는 #Person1#에게 기계를 어떻게 사용하는지 알려주고, 비누가 없다고 말한다. 그녀는 옷을 빨아본 적이 없으므로, #Person2#는 그녀의 엄마가 모든 것을 대..."
497,test_498,스티브와 매튜는 계약이 다음 달에 끝나기 때문에 새로운 집을 찾고 있다. 그들은 다우 부인에게 아파트를 보여줄 것이다.


In [32]:
print(output.iloc[0])

fname                                                                                                   test_0
summary    더슨 씨는 #Person1#에게 사무실 통신은 이메일 통신과 공식 메모로 제한되어 있다고 말한다. #Person2#는 내부와 외부 통신에 적용된다고 설명한다. 그는 또한 새...
Name: 0, dtype: object
